# Cultural Algorithm for Wordle Solver 

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
%load_ext manim
from manim import config
config.verbosity = "ERROR" 

The manim module is not an IPython extension.


In [2]:
# Load word list
with open("words.txt", "r") as f:
    words = [line.strip() for line in f]

print(f"Loaded {len(words)} words")

Loaded 14855 words


## Wordle Game Mechanics

In [3]:
def grey(guessed_word, correct_word, pos):
    """Check if letter at position is grey (not in word)"""
    if guessed_word[pos] in (set(guessed_word) - set(correct_word)):
        return True
    else:
        return False

def green(guessed_word, correct_word, pos):
    """Check if letter at position is green (correct position)"""
    if guessed_word[pos] == correct_word[pos]:
        return True
    else:
        return False

def yellow(guessed_word, correct_word, pos):
    """Check if letter at position is yellow (in word, wrong position)"""
    if grey(guessed_word, correct_word, pos) or green(guessed_word, correct_word, pos):
        return False
    else:
        return True

In [4]:
def color_for_letter(guessed_word, correct_word, pos):
    if green(guessed_word, correct_word, pos):
        return "#6aaa64"  
    elif yellow(guessed_word, correct_word, pos):
        return "#c9b458"  
    else:
        return "#787c7e"  

In [5]:
guessed_word="saleh"
correct_word="parer"

In [6]:
%%manim -ql WordleGuessScene
from manim import *
class WordleGuessScene(Scene):
    def construct(self):
        letters = VGroup()
        for i, char in enumerate(guessed_word):
            rect = Square(side_length=1).set_fill(color_for_letter(guessed_word, correct_word, i), opacity=1).set_stroke(BLACK, 2)
            letter_text = Text(char, font_size=48).move_to(rect.get_center())
            letter_group = VGroup(rect, letter_text)
            letter_group.shift(RIGHT * i * 1.1)
            letters.add(letter_group)
        letters.move_to(ORIGIN)
        for lg in letters:
            self.play(FadeIn(lg, shift=UP*0.5), run_time=0.3)
        self.wait(1)

Manim Community v0.19.0

## Fitness Function (Situational Knowledge)

In [7]:
def fitness(guessed_word, correct_word):
    score = 0
    if guessed_word == correct_word:
        score = 1
    else:
        for pos in range(5):  
            if green(guessed_word, correct_word, pos):
                score += 0.2
            elif yellow(guessed_word, correct_word, pos):
                score += 0.1
    return score

## Position Frequency Analysis (Normative Knowledge)

In [8]:
def position_frequencies(words):
    freq = defaultdict(lambda: [0]*5)

    for word in words:
        for i, ch in enumerate(word):
            freq[ch][i] += 1
    total_words = len(words)
    for ch in freq:
        freq[ch] = [count / total_words for count in freq[ch]]

    return dict(freq)

def score_word(word, pos_freq):
    score = 0.0
    for i, ch in enumerate(word):
        score += pos_freq.get(ch, [0]*5)[i]
    return score

def score_all_words(words, pos_freq):
    scored = {word: score_word(word, pos_freq) for word in words}
    return dict(sorted(scored.items(), key=lambda x: x[1], reverse=True))

## Cultural Algorithm Components

In [9]:
def initialize_population(pop_size, words):
    population = random.sample(words, min(pop_size, len(words)))
    return population

def evaluate_population(population, correct_word):
    return [fitness(ind, correct_word) for ind in population]

def accept_individuals(population, fitness_scores, acceptance_ratio):
    n_accept = int(len(population) * acceptance_ratio)
    indices = np.argsort(fitness_scores)[-n_accept:] 
    return [population[i] for i in indices], [fitness_scores[i] for i in indices]

def update_belief_space(belief_space, accepted_pop, accepted_fit, population, words):
    pos_freq = position_frequencies(words)
    item_frequency = score_all_words(population, pos_freq)

    belief_space['normative']['item_frequency'] = item_frequency
    belief_space['normative']['total_accepted'] = len(accepted_pop)
    
    best_idx = np.argmax(accepted_fit)
    if accepted_fit[best_idx] > belief_space['situational']['best_fitness']:
        belief_space['situational']['best_solution'] = accepted_pop[best_idx]
        belief_space['situational']['best_fitness'] = accepted_fit[best_idx]
        
def evolve_population(population, fitness_scores, words, mutation_rate=0.5):
    pop_size = len(population)
    new_population = []
    words_set = set(words) 
    
    # Elitism
    best_idx = np.argmax(fitness_scores)
    new_population.append(population[best_idx])
    
    while len(new_population) < pop_size:
        idx1, idx2 = np.random.choice(pop_size, 2, replace=False)
        parent1 = population[idx1] if fitness_scores[idx1] > fitness_scores[idx2] else population[idx2]
        
        idx1, idx2 = np.random.choice(pop_size, 2, replace=False)
        parent2 = population[idx1] if fitness_scores[idx1] > fitness_scores[idx2] else population[idx2]
        
        offspring = None
        
        for _ in range(50):
            crossover_point = random.randint(1, 4)
            candidate = parent1[:crossover_point] + parent2[crossover_point:]

            if candidate in words_set:
                offspring = candidate
                break
                
        if offspring is None or np.random.random() < mutation_rate:
            offspring = random.choice(words)  
        
        new_population.append(offspring)
    
    return new_population

In [20]:
def cultural_algorithm_wordle(correct_word, words, pop_size=50, acceptance_ratio=0.2, 
                               influence_ratio=0.3, max_generations=500):
    print("Running Cultural Algorithm for Wordle")
    print(f"Target word: {'*' * len(correct_word)} (hidden)")
    print(f"Population: {pop_size}, Max Generations: {max_generations}")
    print("-" * 60)
    
    # Initialize population
    population = initialize_population(pop_size, words)
    fitness_scores = evaluate_population(population, correct_word)
    
    # Initialize belief space
    pos_freq = position_frequencies(words)
    initial_ranked = score_all_words(words, pos_freq)
    
    belief_space = {
        'normative': {
            'item_frequency': initial_ranked,
            'total_accepted': 0
        },
        'situational': {
            'best_solution': None,
            'best_fitness': float('-inf')
        }
    }
    
    history = []
    solution_found = False
    
    # Evolution loop
    for gen in range(max_generations):
        #elect best individuals
        accepted_pop, accepted_fit = accept_individuals(
            population, fitness_scores, acceptance_ratio
        )
        
        # Update belief space
        update_belief_space(belief_space, accepted_pop, accepted_fit, population, words)
        
        # Evolve
        population = evolve_population(population, fitness_scores, words)
        fitness_scores = evaluate_population(population, correct_word)
        
 
        best_fitness = max(fitness_scores)
        best_idx = np.argmax(fitness_scores)
        best_word = population[best_idx]
        history.append(best_fitness)
        
        if gen % 1 == 0:
            print(f"Generation {gen}: Best Fitness = {best_fitness:.2f}, Best Word = {best_word}")
        
        if best_word == correct_word:
            print(f"\n✓ EXACT SOLUTION FOUND at generation {gen}!")
            print(f"   Word: {best_word}")
            solution_found = True
            break
    
    print("-" * 60)
    

    if solution_found:
        best_solution = correct_word
        best_fitness = 1.0
    else:
        best_solution = belief_space['situational']['best_solution']
        best_fitness = belief_space['situational']['best_fitness']
    
    print(f"\nBest Solution: {best_solution}")
    print(f"Fitness Score: {best_fitness:.2f}")
    print(f"Correct Word: {correct_word}")
    print(f"Match: {best_solution == correct_word}")
    
    if not solution_found:
        print(f"\n⚠ Warning: Reached max generations ({max_generations}) without finding exact match")
    
    return best_solution, history

In [21]:
correct_word = "jazzy"  

best_solution, history = cultural_algorithm_wordle(
    correct_word=correct_word,
    words=words,
    pop_size=50,
    acceptance_ratio=0.2,
    influence_ratio=0.3,
    max_generations=500  
)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Target: {correct_word}")
print(f"Found: {best_solution}")
print(f"Success: {'YES' if best_solution == correct_word else 'NO'}")
print(f"Generations: {len(history)}")
print(f"Final Fitness: {history[-1]:.2f}")

Running Cultural Algorithm for Wordle
Target word: ***** (hidden)
Population: 50, Max Generations: 500
------------------------------------------------------------
Generation 0: Best Fitness = 0.50, Best Word = pakay
Generation 1: Best Fitness = 0.50, Best Word = pakay
Generation 2: Best Fitness = 0.50, Best Word = pakay
Generation 3: Best Fitness = 0.50, Best Word = pakay
Generation 4: Best Fitness = 0.50, Best Word = pakay
Generation 5: Best Fitness = 0.50, Best Word = pakay
Generation 6: Best Fitness = 0.50, Best Word = pakay
Generation 7: Best Fitness = 0.50, Best Word = pakay
Generation 8: Best Fitness = 0.50, Best Word = pakay
Generation 9: Best Fitness = 0.50, Best Word = pakay
Generation 10: Best Fitness = 0.60, Best Word = gauzy
Generation 11: Best Fitness = 0.60, Best Word = gauzy
Generation 12: Best Fitness = 0.60, Best Word = gauzy
Generation 13: Best Fitness = 0.60, Best Word = gauzy
Generation 14: Best Fitness = 0.60, Best Word = gauzy
Generation 15: Best Fitness = 0.60, 